# Misson 3

## 기능요구사항
- IMF에서 제공하는 국가별 GDP를 구하세요.

- 국가별 GDP를 확인할 수 있는 테이블을 만드세요.

- 해당 테이블에는 GDP가 **높은 국가들이 먼저** 나와야 합니다.

- GDP의 단위는 1B USD이어야 하고 **소수점 2자리**까지만 표시해 주세요.

- IMF에서 매년 2회 이 자료를 제공하기 때문에 정보가 갱신되더라도 **해당 코드를 재사용**해서 정보를 얻을 수 있어야 합니다.

## 프로그래밍 요구 사항
- **주석**을 사용해서 설명을 추가하고 
  
- 함수를 만들어서 가독성과 재사용성을 높이세요.
  
- 추출 한 정보는 '**Countries_by_GDP.json**'라는 이름의 JSON 화일 포맷으로 저장해야 합니다.
  
- 필요한 모든 작업을 수행하는 '**etl_project_gdp.py**' 코드를 작성하세요.

## 화면 출력
- **GDP가 100B USD이상**이 되는 국가만을 구해서 화면에 출력해야 합니다.

- 각 **Region별로 top5** 국가의 **GDP 평균**을 구해서 화면에 출력해야 합니다.

## ETL 프로세스
- **ETL 프로세스**에 따라 코드를 작성하고 각 단계의 **시작과 끝을 로그**에 기록하세요.
- 

- 이 모든 처리 과정은 'etl_project_log.txt'라는 로그 화일에 기록되어야 합니다. (로그 화일은 매번 다시 생성하는 것이 아니라 **기존 화일에 append**되어야 합니다.)

- log는 "**time, log**" 형식으로 기록하세요. 시간은 'Year-Monthname-Day-Hour-Minute-Second' 포맷에 맞게 표시하세요.

In [61]:
import requests
from bs4 import BeautifulSoup# 크롤링
import pandas as pd # 데이터 처리
import json # to json file
import datetime # logging
import sqlite3 # DB
import re
import time, datetime

In [67]:
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'

In [68]:
response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
else:
    print(response.status_code)
    

In [104]:
type(html)

str

In [12]:
''' 
Table Head Structure:
        | IMF               | WB                | UN                | 
Country |Forecast   | Year  |Forecast   | Year  |Forecast   | Year  |
        | :TODO     | :TODO |
----------------------------------------------------------------------       
Table Row Structure:
td      | td        | td    | ...
'''

' Table Head\n        | IMF   \nCountry |\n'

In [73]:
table = soup.select('table.wikitable > tbody > tr:nth-child(n+4)')#.wikitable sortable sticky-header-multi static-row-numbers jquery-tablesorter')
# tr, td 등의 tag가 변수 취급이 가능하구나
gdp_rows = [[td.get_text(strip=True) for td in tr.find_all('td')] for tr in table]

print(gdp_rows)

[['United States', '30,337,162', '2025', '27,360,935', '2023', '25,744,100', '2022'], ['China', '19,534,894', '[n 1]2025', '17,794,782', '[n 3]2023', '17,963,170', '[n 1]2022'], ['Germany', '4,921,563', '2025', '4,456,081', '2023', '4,076,923', '2022'], ['Japan', '4,389,326', '2025', '4,212,945', '2023', '4,232,173', '2022'], ['India', '4,271,922', '2025', '3,549,919', '2023', '3,465,541', '2022'], ['United Kingdom', '3,730,261', '2025', '3,340,032', '2023', '3,089,072', '2022'], ['France', '3,283,429', '2025', '3,030,904', '2023', '2,775,316', '2022'], ['Italy', '2,459,597', '2025', '2,254,851', '2023', '2,046,952', '2022'], ['Canada', '2,330,308', '2025', '2,140,086', '2023', '2,137,939', '2022'], ['Brazil', '2,307,162', '2025', '2,173,666', '2023', '1,920,095', '2022'], ['Russia', '2,195,708', '2025', '2,021,421', '2023', '2,240,422', '2022'], ['South Korea', '1,947,133', '2025', '1,712,793', '2023', '1,673,916', '2022'], ['Australia', '1,881,140', '2025', '1,723,827', '2023', '1,77

In [74]:
re.sub(r',', '', '10,000')

'10000'

In [ ]:
country = []
gdp = []
year = []
no_juseok = lambda s: re.sub(r'\[.*?\]', '', s)
str_to_num = lambda s: int(re.sub(r',', '', s))

for row in gdp_rows:
    print(row)
    country.append(no_juseok(row[0]))
    
    if row[1] == '—': gdp.append(-1)
    else: gdp.append(str_to_num(no_juseok(row[1])))
    
    if row[2] == '—': year.append(-1)
    else: year.append(no_juseok(row[2]))

In [98]:
print(len(country), len(gdp), len(year)) # checked!

209 209 209


In [ ]:
gdp = [round(i / 1000, 2) for i in gdp]
GDP_Table = pd.DataFrame({'Country': country, 'GDP': gdp, 'Year': year}).sort_values('GDP', ascending=False, inplace=True)

In [103]:
GDP_Table = GDP_Table[GDP_Table['GDP'] >= 100]

## Prototype

In [30]:
import requests
from bs4 import BeautifulSoup# 크롤링
import pandas as pd # 데이터 처리
import json # to json file
import datetime # logging
import sqlite3 # DB
import re
import time, datetime


class Extract:
    WIKI_URL = ''
    IMF_URL = ''
    
    def __init__(self):
        self.WIKI_URL = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29'
        self.IMF_URL = ''
    
    def url_validation_check(self, url:str):
        '''
        URL 유효성 검증
        Param: url: str
        return: html text
        '''
        response = requests.get(url)

        if response.status_code == 200:
            return response.text
        else:
            print(response.status_code)  
            return False      
    
    def collect_data(self, html: str):
        '''
        원본 데이터 수집
        Param: html: str
        return: gdp_rows: List 
        # 국가별 자료입니다. 형식: [[국가명, GDP, 연도, GDP, 연도, GDP, 연도] ... ] 
        '''
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.select('table.wikitable > tbody > tr:nth-child(n+4)')#.wikitable sortable sticky-header-multi static-row-numbers jquery-tablesorter')
        # tr, td 등의 tag가 변수 취급이 가능하구나
        gdp_rows = [[td.get_text(strip=True) for td in tr.find_all('td')] for tr in table]

        return gdp_rows


class Transform:
    raw_data = []
    def __init__(self, raw_data:list):
        '''
        데이터를 가공하기 위해 원본 데이터를 가져옵니다.
        '''
        self.raw_data = raw_data
    
    def refine_data(self):
        '''
        데이터를 정제합니다.
        링크를 지우고 문자열로 된 숫자를 정수로 변환합니다.
        '-'로 표기된 데이터는 -1 을 기입합니다.
        
        return: country: List, gdp: List, year: List
        '''
        country = []
        gdp = []
        # year = []
        
        no_juseok = lambda s: re.sub(r'\[.*?\]', '', s)
        str_to_num = lambda s: int(re.sub(r',', '', s))
        
        for i, row in enumerate(self.raw_data):

            if len(row) < 2: 
                print(f"Data Error in line {i}")
                
            country.append(no_juseok(row[0]))
            
            if row[1] == '—': 
                gdp.append(-1)
            else: 
                gdp.append(str_to_num(no_juseok(row[1])))
        
        return country, gdp
        
    def make_DataFrame(self, country:list, gdp:list):
        '''
        가공된 데이터로 데이터프레임을 생성합니다.
        GDP에 대해 내림차순 정렬되어 있고 GDP가 $100B 이상인 국가만 포함됩니다.

        Param: country: List, gdp: List, year: List
        return: GDP_data: pd.DataFrame
        '''
        GDP_data = pd.DataFrame({'Country': country, 'GDP_USD_Bilion': gdp})
        GDP_data.sort_values('GDP_USD_Bilion', ascending=False, inplace=True)
           
        GDP_data['GDP_USD_Bilion'] = GDP_data['GDP_USD_Bilion'].apply(lambda x: round(x / 1000, 2))
        
        
        return GDP_data[GDP_data['GDP_USD_Bilion'] >= 100]
    
class Load:
    frame = None
    def __init__(self, frame):
        '''
        데이터프레임을 저장하기 위해 불러옵니다.
        '''
        self.frame = frame
    
    def save_json(self,):
        '''
        데이터프레임을 JSON 형식으로 저장합니다.
        '''
        try:
            filename = 'Countries_by_GDP.json'
            self.frame.to_json(filename, indent=4)
            print(f"Updated: {filename}")
        
        except Exception as e:
            print(f"Error processing {filename}: {e}")

## Test Code

In [31]:
e = Extract()
print(e.WIKI_URL)
if html := e.url_validation_check(e.WIKI_URL):
    raws = e.collect_data(html)
t = Transform(raws)
table = t.make_DataFrame(*t.refine_data())
table

https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29


,Country,GDP_USD_Bilion
0,United States,30337.16
1,China,19534.89
2,Germany,4921.56
3,Japan,4389.33
4,India,4271.92
...,...,...
68,Uzbekistan,112.65
69,Guatemala,112.37
70,Oman,109.99
71,Bulgaria,108.42


In [32]:
try:
    with sqlite3.connect('World_Economies.db') as conn:
        print(f"Opened SQLite database with version {sqlite3.sqlite_version} successfully.")

except sqlite3.OperationalError as e:
    print("Failed to open database:", e)
    
table.to_sql('Countries_by_GDP', conn)

Opened SQLite database with version 3.39.5 successfully.


72

In [87]:
import logging, logging.config
from functools import wraps

logger = logging.getLogger()
if not logger.handlers:
    # 사용자 정의 Formatter
    formatter = logging.Formatter(
        fmt='%(asctime)s, %(message)s',
        datefmt='%Y-%B-%d-%H-%M-%S'  # 원하는 날짜 포맷
    )

    # 핸들러 생성 및 설정
    handler = logging.FileHandler('etl_project_log.txt')
    handler.setFormatter(formatter)

    # 로거 설정
    logger.setLevel(logging.INFO)
    logger.addHandler(handler)
# # 기존 핸들러 제거
# for handler in logger.handlers[:]:
#     logger.removeHandler(handler)

# 테스트 로그 메시지
logger.info("This is a test log message.")

def logging_time(func):         
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = datetime.datetime.now()
        # formatted_start = start_time.strftime("%Y-%B-%d-%H-%M-%S")

        logging.info(f"Started '{func.__name__}'")
        
        result = func(*args, **kwargs)
        
        end_time = datetime.datetime.now()
        # formatted_end = end_time.strftime("%Y-%B-%d-%H-%M-%S")

        logging.info(f"Finished '{func.__name__}'")
        # logging.info(f"Execution time for '{func.__name__}': {end_time - start_time}")
        return result
    return wrapper

In [88]:
@logging_time
def sample_function(duration):
    """샘플 함수: 주어진 시간(초) 동안 대기"""
    import time
    time.sleep(duration)
    return "Done!"
result_sample = sample_function(2)
print(result_sample)

Done!


In [ ]:
Algeria
Angola
Benin
Botswana
Burkina Faso
Burundi
Cameroon
Cape Verde
Central African Republic
Chad
Comoros
Congo, Dem. Rep. of
Congo, Republic of
Côte d'Ivoire
Djibouti
Egypt
Equatorial Guinea
Eritrea
Eswatini
Ethiopia
Gabon
Gambia, The
Ghana
Guinea
Guinea-Bissau
Kenya
Lesotho
Liberia
Libya
Madagascar
Malawi
Mali
Mauritania
Mauritius
Mayotte
Morocco
Mozambique
Namibia
Niger
Nigeria
Reunion
Rwanda
Saint Helena
São Tomé and Príncipe
Senegal
Seychelles
Sierra Leone
Somalia
South Africa
South Sudan
Sudan
Tanzania
Togo
Tunisia
Uganda
Zambia
Zimbabwe

# 추가 요구 사항
## 코드를 수정해서 아래 요구사항을 구현하세요.
- 추출한 데이터를 데이터베이스에 저장하세요. 'Countries_by_GDP'라는 테이블명으로 'World_Economies.db'라는 데이터 베이스에 저장되어야 합니다. <br>해당 테이블은 'Country', 'GDP_USD_billion'라는 어트리뷰트를 반드시 가져야 합니다.

- 데이터베이스는 sqlite3 라이브러리를 사용해서 만드세요.

- 필요한 모든 작업을 수행하는 'etl_project_gdp_with_sql.py' 코드를 작성하세요.

## 화면 출력
- SQL Query를 사용해야 합니다.

- GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.

- 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다.